<h1>LaVague: Quick-tour guide</h1>

### Installation

We can install everything we need to get started with LaVague using our `setup.sh` script.

We'll also download the config and instruction files we will use in this demo.

> This downloads the webdriver needed to leverage Selenium to interact with the browser, as well as the LaVague package.

In [ ]:
# Run the LaVague setup script
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/setup.sh
!sudo bash setup.sh

# Download files needed for demo to current repo
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/examples/configurations/api/openai_api.py
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/examples/instructions/huggingface.yaml

### OpenAI API key

By default, LaVague uses OpenAI GPT3.5 and an OpenAI light embedder. Please make sure you've set the `OPENAI_API_KEY` environment variable with your OpenAI key, or if you're running this in our Google Colab, set it in the command below.

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = # ADD YOUR OPENAI KEY HERE

### LaVague Launch

Let's now run the `lavague [OPTIONS] launch` command!

- The `--instructions` or `-i` option accepts a text file containing: the URL of the website we will interact with & the instructions for the actions we wish to automate
-  The `--config` or `-c` option with a Python file which can be used to set a desired LLM, embedder etc.

For more info on these, see the [customization guide](https://docs.lavague.ai/en/latest/docs/get-started/customization/)!

In [ ]:
!lavague -i huggingface.yaml -c openai_api.py launch

You can now click on the public (if you are using Google Colab) or local URL to open your interactive LaVague demo.

⚠️ You'll need to interact with this demo:

1) Click on the URL textbox and press enter. This will show a screenshot of your inital page.

<img src="https://raw.githubusercontent.com/lavague-ai/LaVague/main/docs/assets/launch_1_openai_py.png" />

2) Then select an instruction or write your own, and again click within the instruction textbox and press enter.

At this point, your Python code leveraging Selenium is generated by the LLM & executed. You will see an updated image of the website after the code is executed.

<img src="https://raw.githubusercontent.com/lavague-ai/LaVague/main/docs/assets/launch_2_openai_py.png" />

### LaVague Build

We can use `lavague [OPTIONS] build` command to generate a file containing this automation code without launching an interactive demo.

In [ ]:
!lavague -i huggingface.yaml -c openai_api.py build

This creates a script in your current directory named `hf_openai_api_gen.py` - a combination of the two input file names separated by a `_` character with a `_gen` suffix.

We can now inspect the code and execute it locally!

<img src="https://raw.githubusercontent.com/lavague-ai/LaVague/main/docs/assets/build_openai_py.png"/>


If you have any further questions, join us on the LaVague Discord [here](https://discord.com/invite/SDxn9KpqX9).